***IMPORTING LIBRARIES***

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
import torchvision
import numpy as np
import torchvision.transforms as t
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import torchvision.models as model
import tensorflow as tf
from PIL import Image as im
import cv2
#from sklearn.metrics import accuracy_score

In [4]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device= ", device)

device=  cuda


***ASSIGNING VARIABLES TO THE PATH***

In [5]:
train_path="/content/drive/MyDrive/chest_xray/train"
test_path="/content/drive/MyDrive/chest_xray/test"
val_path="/content/drive/MyDrive/chest_xray/val"
plot_path="/content/drive/MyDrive/chest_xray/plot_path"
snapshot_path="/content/drive/MyDrive/chest_xray/snapshot_path"

***DATA AUGMENTATION***

In [6]:
model_name='ResNet18'
batch_s = 16
transform=t.Compose([t.Resize((224,224)),
                     t.RandomCrop((224,224)),
                     t.RandomHorizontalFlip(),
                     t.RandomVerticalFlip(),
                     t.RandomAffine(degrees=(-180,180), translate=(0.1,0.1), scale=(0.9,1.1), shear=(-5,5)),
                     t.ToTensor()])
dset_train=torchvision.datasets.ImageFolder(root=train_path,transform=transform)

test_trans=t.Compose([t.Resize((224,224)),t.ToTensor()])
dset_test=torchvision.datasets.ImageFolder(root=test_path,transform=test_trans)
dset_val=torchvision.datasets.ImageFolder(root=val_path,transform=test_trans)


train_loader=torch.utils.data.DataLoader(dset_train,batch_size=batch_s,shuffle=True,num_workers=16)#,drop_last=True)
val_loader=torch.utils.data.DataLoader(dset_val,batch_size=batch_s,shuffle=False,num_workers=16)#,drop_last=True)
test_loader=torch.utils.data.DataLoader(dset_test,batch_size=batch_s,num_workers=16)#, drop_last=True)


num_classes = 2

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


***DEVELOPING THE MODEL***

In [7]:
models = torchvision.models.resnet18(pretrained=False)


class MyModel(nn.Module):
  def __init__(self):
    super(MyModel, self).__init__()
    img_modules = list(models.children())[:-1]
    self.ModelA = nn.Sequential(*img_modules)
    self.Linear1 = nn.Linear(1024, 256)
    self.relu = nn.ReLU()
    self.Linear2 = nn.Linear(256, 2)
    self.Linear3 = nn.Linear(512, 2, bias = True)

  def forward(self, x):
    x = self.ModelA(x)
    x1 = torch.flatten(x, 1)
    x2 = self.Linear3(x1)


    return  x1, x2


***ASSIGNING PARAMS CRITERION AND OPTIMIZER***

In [8]:
from torch.optim.optimizer import Optimizer
from torch import optim
net = MyModel()
net=net.cuda()
criterion=nn.CrossEntropyLoss()
params = net.parameters()
optimizer=torch.optim.Adam(net.parameters(), lr = 0.001)
model_name1 = 'ResNet18_last'

***LOADING THE MODEL***

In [9]:
load_model=snapshot_path+'/model_'+model_name+'.pth'
loaded_flag=False
if os.path.exists(load_model):
    checkpoint=torch.load(load_model)
    net.load_state_dict(checkpoint['model_state'])
    optimizer.load_state_dict(checkpoint['optimizer_state'])
    print("model loaded successfully")
    print('starting training after epoch: ',checkpoint['epoch'])
    loaded_flag=True
    

model loaded successfully
starting training after epoch:  84


***PLOT FUNCTION***

In [10]:
def plot(val_loss,train_loss):
    plt.title("Loss after epoch: {}".format(len(train_loss)))
    plt.xlabel("epoch")
    plt.ylabel("loss")
    plt.plot(list(range(len(train_loss))),train_loss,color="r",label="Train_loss")
    plt.plot(list(range(len(val_loss))),val_loss,color="b",label="Validation_loss")
    plt.legend()
    plt.savefig(os.path.join(plot_path,"loss_"+model_name+".png"))
    
    
    plt.close()
  


In [11]:
val_interval=1
min_loss=99999
val_loss_gph=[]
train_loss_gph=[]

if loaded_flag:
    min_loss=checkpoint['loss']
    val_loss_gph=checkpoint["val_graph"]
    train_loss_gph=checkpoint["train_graph"]

***TRAINING LOOP***

In [12]:
def train(epoch=5):
  i=0
  global min_loss
  flag=True
  
  while i+1<=epoch and flag:
    print("Epoch {}".format(i+1 if not loaded_flag else i+1+checkpoint['epoch']))
    train_loss=0.0
    i+=1
    data1 = []
    correct=total=0

    for (image,label) in train_loader:
      net.train()
      optimizer.zero_grad()
      outputs1, outputs2=net(image.cuda())
      loss=criterion(outputs2 ,label.cuda())
      loss.backward()
      optimizer.step()
      train_loss+=loss.item()*image.size(0)
      _, predicted = torch.max(outputs2.data, 1)
      total += label.size(0)
      correct += (predicted == label.cuda()).sum().item()
    print("Train accuracy", (100*correct/total))
    train_loss_gph.append(train_loss/len(dset_train))
    

    if (i+1)%val_interval==0 or (i+1)==epoch:
        net.eval()
        with torch.no_grad():
          val_loss=0
          correct=total=0
          for (img_v,lab_v ) in val_loader:

            output_v1, output_v2=net(img_v.cuda())
            val_loss+=criterion(output_v2,lab_v.cuda()).item()*img_v.size(0)
            _, predicted = torch.max(output_v2.data, 1)
            total += lab_v.size(0)
            correct += (predicted == lab_v.cuda()).sum().item()
          print("Val accuracy", (100*correct/total))
          val_loss_gph.append(val_loss/len(dset_val))
        
          if val_loss<min_loss:
            state={
                "epoch":i if not loaded_flag else i+checkpoint['epoch'],
                "model_state":net.cpu().state_dict(),
                "optimizer_state":optimizer.state_dict(),
                "loss":min_loss,
                "train_graph":train_loss_gph,
                "val_graph":val_loss_gph,
            }
            
            min_loss=val_loss
            torch.save(state,os.path.join(snapshot_path,"model_"+model_name+'.pth'))
            net.cuda()
          print("validation loss : {:.6f} ".format(val_loss/len(dset_val)))
    plot(val_loss_gph,train_loss_gph)
    print("Train loss : {:.6f}".format(train_loss/len(dset_train)))
    if i==epoch:
      flag=False
      break
  
train(85)

Epoch 85


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train accuracy 97.81441717791411
Val accuracy 68.75
validation loss : 1.093890 
Train loss : 0.058983


***RUNNING THE MODEL ON TEST DATA AND PRINTING ACCURACY FOR EACH CLASS***

In [13]:
net=net.eval()
correct = 0
total = 0
data1 = []
with torch.no_grad():
      for data in test_loader:
          images, labels = data
          labels=labels.cuda()
          outputs1, outputs2 = net(images.cuda())
          #data1.append(outputs1)
          _, predicted = torch.max(outputs2.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
      100 * correct / total))

#print("ACCURACY ON TEST DATASET ON NORMAL")

net=net.eval()
correct = 0
total = 0
data2 = []
var_1 =0
var_2 =0
with torch.no_grad():
      for data in test_loader:
          images, labels = data
          labels=labels.cuda()
          outputs1, outputs2 = net(images.cuda())
          _, predicted = torch.max(outputs2.data, 1)
          y= (labels==0).sum().item()
          
          if y>0:
            var_1 += ((predicted == labels) * (labels == 0)).sum().item()
            var_2 += y
print('Accuracy of the network on the test-NORMAL images: %d %%' % (
      100 * var_1 / var_2))


net=net.eval()
correct = 0
total = 0
data2 = []
var_1 =0
var_2 =0
with torch.no_grad():
      for data in test_loader:
          images, labels = data
          labels=labels.cuda()
          outputs1, outputs2 = net(images.cuda())
          _, predicted = torch.max(outputs2.data, 1)
          y= (labels==1).sum().item()
          
          if y>0:
            var_1 += ((predicted == labels) * (labels == 1)).sum().item()
            var_2 += y
            
print('Accuracy of the network on the test-PNEUMONIA images: %d %%' % (
      100 * var_1 / var_2))




/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Accuracy of the network on the test images: 88 %
Accuracy of the network on the test-NORMAL images: 71 %
Accuracy of the network on the test-PNEUMONIA images: 98 %


***MISCLASSIFICATION OF IMAGES***

In [17]:
incorrect_examples = []
incorrect_labels = []
net = net.eval()

for data,target in test_loader:

  data , target =  data.to(device=device), target.to(device=device)
  target =target.cuda()
  outputs1, outputs2 = net(data.cuda())
  _, pred = torch.max(outputs2.data, 1)
  idxs_mask = ((pred == target.view_as(pred))==False).view(-1)
  if len(data[idxs_mask]) > 0:
    incorrect_examples.append(data[idxs_mask][0].squeeze().cpu().permute(2,1,0).numpy())
    incorrect_labels.append(target[idxs_mask].cpu().numpy()) #the corresponding target to the misclassified image   

x=incorrect_examples[0]
x *= 255
cv2.imwrite("MISCLASSIFIED IMAGE 1.png", x )

x1=incorrect_examples[1]
x1 *= 255
cv2.imwrite("MISCLASSIFIED IMAGE 2.png", x1 )


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


True